# Entanglement Testing

In [1]:
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.ibmq.managed import IBMQJobManager

# Loading your IBM Q account(s)
import qiskit
from qiskit import IBMQ
#IBMQ.save_account("9e797fbe9a7b252773d8ecb4b561f09712835b125573c97213c2b3c8c19bdd2aae7309d90d9559961043ee89a263cc1f9a61c97b75c7441ee6716e1eda9af854")
provider = IBMQ.load_account()

import matplotlib.pyplot as plt
import numpy as np

# custom code
import context
from device_independent_test import entanglement

In [2]:
# preparing a maximally entangled state
circuit = entanglement.create_entangled_circuit()
circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [3]:
# measure in Z basis for qubit 0, in W basis for qubit 1
entanglement.measure_in_ZW(circuit)
circuit.draw()

┌───┐                          ░ ┌─┐   
   q_0: ┤ H ├──■───────────────────────░─┤M├───
        └───┘┌─┴─┐┌───┐┌───┐┌───┐┌───┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ S ├┤ H ├┤ T ├┤ H ├─░──╫─┤M├
             └───┘└───┘└───┘└───┘└───┘ ░  ║ └╥┘
meas_0: ══════════════════════════════════╩══╬═
                                             ║ 
meas_1: ═════════════════════════════════════╩═

In [4]:
simulator = Aer.get_backend("qasm_simulator")
shotnum = 10000

In [5]:
result = execute(circuit, backend=simulator, shots=shotnum).result()
#measurement = entanglement.measure_in_ZW(circuit)
#measurement = entanglement.measure_in_ZW(circuit)
#measurement = entanglement.measure_in_ZW(circuit)
#circuit.draw()

In [6]:
#from qiskit.tools.visualization import plot_histogram
stats = result.get_counts(circuit)
stats

{'10': 743, '11': 4350, '01': 748, '00': 4159}

In [7]:
expectation_XW = (stats['11']+stats['00']-stats['10']-stats['01'])/shotnum
expectation_XW

0.7018